In [1]:
from pyelpa import DistributedMatrix
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-2x2x2/"

In [2]:
def print_2d_matrix(matrix, decimals=3):
    for row in matrix:
        print(" ".join(f"{val:9.{decimals}f}" for val in row))

In [3]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [4]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# WFN data
wfndata = wfn2py(dirname+'WFN.h5')
wfnqdata = wfn2py(dirname+'WFNq.h5')

# RHO data
rho = inp.read_rho(dirname+"RHO")

# Vxc data
vxc = inp.read_vxc(dirname+"vxc.dat")

In [5]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 8/8 [00:00<00:00, 15932.78it/s]


In [6]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)


        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv, indices_gspace_sorted)
        epsilon.l_epsinv[i_q] = epsinv
        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))

        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")

    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)
calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 8/8 [00:00<00:00, 112.12it/s]

Standard deviation exceeded 1e-16 tolerance: 2.5494915419483335e-15, for i_q:0


In [7]:
from qtm.gw.sigma import Sigma

sigma = Sigma.from_data(
    wfndata=wfndata,
    wfnqdata=wfnqdata,
    sigmainp=sigmainp,
    epsinp=epsinp,
    l_epsmats=epsilon.l_epsinv,
    rho=rho,
    vxc=vxc,
)

Vcoul calculation for qpts: 100%|██████████| 8/8 [00:00<00:00, 12826.62it/s]


vcoul: Vcoul:
        * gspace = <qtm.gspace.gspc.GSpace object at 0x781c78b66690>
        * qpts = <qtm.gw.core.QPoints object at 0x781c785b01d0>
        * bare_coulomb_cutoff = 2.0
        * avgcut = 1e-05
        * l_gspace_q = <class 'list'> of length 8
        * vcoul = <class 'list'> of length 8
        * N_SAMPLES = 2500000.0
        * N_SAMPLES_COARSE = 250000.0
        * SEED = 5000
        


Vcoul calculation for qpts:   0%|          | 0/8 [00:00<?, ?it/s]

Vcoul calculation for qpts: 100%|██████████| 8/8 [00:05<00:00,  1.35it/s]


In [8]:
# Calculate the quasiparticle energies.
sigma.print_condition = False
results_dict = sigma.calculate_static_cohsex()

k_indices = sigma.l_k_indices
num_bands = len(results_dict[k_indices[0]]["Eqp1"])

eqp1_array = np.zeros((len(k_indices), num_bands))
for i, k_idx in enumerate(k_indices):
    eqp1_array[i, :] = results_dict[k_idx]["Eqp1"]

Sigma_CH_Static_Exact: 100%|██████████| 8/8 [00:00<00:00, 92.97it/s]


In [9]:
print("Quasiparticle energies (Eqp1):")
print_2d_matrix(eqp1_array, decimals=6)

Quasiparticle energies (Eqp1):
-6.760997  7.302883  7.303055  7.299997 10.848427 10.848878 10.844457 15.054266
-3.850785 -1.139382  5.640210  5.636517 10.817296 11.579493 11.577671 15.093000
-3.850713 -1.139732  5.640509  5.636530 10.817266 11.579541 11.577562 15.093033
-1.803499 -1.802659  3.380840  3.376889  8.420505  8.421883 19.697087 19.697217
-3.850688 -1.139823  5.640417  5.636727 10.817360 11.579803 11.577952 15.093085
-1.803682 -1.802751  3.380909  3.377123  8.421066  8.421617 19.696966 19.697085
-1.803722 -1.802455  3.380553  3.377231  8.420957  8.421618 19.697046 19.697046
-3.851600 -1.141514  5.639899  5.640203 10.816857 11.582074 11.581970 15.093132


In [10]:
from kernel import KernelMtxEl

q0 = [0.001, 0.001, 0.001]
l_qpts = np.array(epsinp.qpts)
l_qpts[0] *= 0
qpts = QPoints.from_cryst(wfndata.kpts.recilat, None, *l_qpts)

kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    q0=q0,
    l_epsmats=epsilon.l_epsinv,
    parallel=True,
)

In [11]:
data = kernelclass.kernel_mtxel()
exc = data["exc"]
head = data["head"]
body = data["body"]
wings = data["wings"]

In [12]:
numq = kernelclass.qpts.numq
numk = kernelclass.kpts.numk

In [13]:
from intkernel import InterpMtxEl

InterpClass = InterpMtxEl.from_BGW(
    wfn_finedata=wfndata,
    wfn_coarsedata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    kernel=kernelclass,
)

In [14]:
energyval = InterpClass.interp_energy(eqp1_array, 'val')
energycon = InterpClass.interp_energy(eqp1_array, 'con')

print("Interpolated quasiparticle energies (valence):")
print_2d_matrix(energyval, decimals=6)

print("Interpolated quasiparticle energies (conduction):")
print_2d_matrix(energycon, decimals=6)


Interpolated quasiparticle energies (valence):
 7.299997  7.303055  7.302883 -6.760997
 5.636517  5.640210 -1.139382 -3.850785
 5.636530  5.640509 -1.139732 -3.850713
 3.376889  3.380840 -1.802659 -1.803499
 5.636727  5.640417 -1.139823 -3.850688
 3.377123  3.380909 -1.802751 -1.803682
 3.377231  3.380553 -1.802455 -1.803722
 5.640203  5.639899 -1.141514 -3.851600
Interpolated quasiparticle energies (conduction):
10.848427 10.848878 10.844457 15.054266
10.817296 11.579493 11.577671 15.093000
10.817266 11.579541 11.577562 15.093033
 8.420505  8.421883 19.697087 19.697217
10.817360 11.579803 11.577952 15.093085
 8.421066  8.421617 19.696966 19.697085
 8.420957  8.421618 19.697046 19.697046
10.816857 11.582074 11.581970 15.093132


In [15]:
is_equal_val = np.allclose(energyval, np.flip(eqp1_array[:, :4], axis = -1))
is_equal_con = np.allclose(energycon, eqp1_array[:, 4:])

print(f"Interpolated quasiparticle energies (valence) are equal to the original: {is_equal_val}")
print(f"Interpolated quasiparticle energies (conduction) are equal to the original: {is_equal_con}")

Interpolated quasiparticle energies (valence) are equal to the original: True
Interpolated quasiparticle energies (conduction) are equal to the original: True


In [16]:
fine_kernel = InterpClass.interp_kernel(head, wings, body, exc, sigma.vcoul)
fine_kernel_selected = fine_kernel[:, :, 0, 0, 0, 0]
print(f"\n Selected part of fine_kernel is")

print_2d_matrix(fine_kernel_selected, decimals=6)


 Selected part of fine_kernel is
0.040035-0.000000j -0.024301+0.092700j -0.053412+0.017990j -0.030163-0.054210j -0.089081-0.136263j 0.007064+0.024259j 0.010095-0.039685j 0.002174+0.000260j
-0.024301-0.092700j 0.102230+0.000000j 0.026699+0.040549j -0.147779+0.141689j 0.031831-0.035217j -0.172294+0.103509j 0.029551-0.000177j -0.001129-0.008178j
-0.053412-0.017990j 0.026699-0.040549j 0.102231-0.000000j 0.036727+0.199586j -0.012200-0.047077j -0.001228+0.029526j 0.110343-0.170478j 0.000301+0.000144j
-0.030163+0.054210j -0.147779-0.141689j 0.036727-0.199586j 0.080787+0.000000j -0.029473+0.002151j 0.013073+0.002983j -0.009758-0.009208j -0.012208+0.016893j
-0.089081+0.136263j 0.031831+0.035217j -0.012200+0.047077j -0.029473-0.002151j 0.102230-0.000000j -0.192361-0.058470j 0.138536+0.150901j -0.005205+0.006196j
0.007064-0.024259j -0.172294-0.103509j -0.001228-0.029526j 0.013073-0.002983j -0.192361+0.058470j 0.080753+0.000000j -0.011549-0.006803j -0.000217+0.000636j
0.010095+0.039685j 0.029551+

In [17]:
HBSE = InterpClass.construct_HBSE(fine_kernel, energyval, energycon)
print(f"\n HBSE is")
print_2d_matrix(HBSE, decimals=6)


 HBSE is
3.588465-0.000000j 0.009585-0.026396j 0.017910+0.026083j 0.030228-0.009156j 0.000575+0.000433j -0.027047+0.017654j -0.001576+0.036200j 0.000344+0.000125j -0.000985-0.000167j -0.034799+0.061545j -0.011566-0.024915j -0.003134+0.000397j 0.000726+0.002965j -0.045963-0.004900j 0.011639-0.013051j 0.002087+0.003530j -0.024301+0.092700j 0.039497-0.008459j -0.001011-0.000006j -0.001098-0.006215j 0.192036-0.033727j -0.008745-0.011682j 0.000811+0.001797j 0.020137-0.013947j 0.033264-0.003141j 0.017488+0.027779j 0.000177+0.000490j -0.031070+0.018044j 0.000880+0.000494j -0.000001+0.000006j 0.000882-0.003277j -0.000172-0.000014j -0.053412+0.017990j 0.031512-0.036665j 0.000568+0.000014j 0.000738+0.004403j 0.228136-0.011232j -0.008399+0.005549j -0.002053-0.000637j -0.003488+0.031834j -0.018672+0.019470j -0.008638+0.037137j 0.000329-0.000161j 0.013737+0.018150j 0.000203+0.000552j -0.000000-0.000000j -0.000006+0.002726j 0.000094-0.000018j -0.030163-0.054210j 0.000297-0.000671j -0.018707+0.02748

In [19]:
from diag import diag_elpa

eigval_elpa, eigvec_elpa = diag_elpa(HBSE)
eigval_numpy, eigvec_numpy = np.linalg.eigh(HBSE)

print(f"\n First ten eigenvalues from ELPA are")
print(eigval_elpa[:10])
print(f"\n First ten eigenvalues from numpy are")
print(eigval_numpy[:10])




 First ten eigenvalues from ELPA are
[3.40685152 3.40944454 3.40983593 3.54746406 3.54761426 3.5527982
 3.56810003 3.5765195  3.57686681 4.78473765]

 First ten eigenvalues from numpy are
[3.40684974 3.40945343 3.40982313 3.54746106 3.54760155 3.55281361
 3.56810667 3.57652016 3.57687365 4.78473588]
